# Analysis of A. thaliana RNA-Seq data with pyrpipe 
Use A thaliana public RNA-Seq data to assemble transcripts.

In [3]:
from pyrpipe import sra,mapping,assembly,qc,tools
#First get the srr accessions of the runs. For this one can use the python package pysradb or R package sradb
#i will consider following randomly selected accessions
athalRuns=['SRR976159','SRR978411','SRR978410','SRR971778','SRR1058116','SRR1058118','SRR1058121','SRR1058110','SRR1058120','SRR1058117','SRR1104134','SRR1104133','SRR1104135','SRR1104136','SRR1105825']

## Download data and create SRA objects
First we can donload all data to disk and save SRA objects to memory.

In [4]:
#set your working directory if you don't wan to use the 
workingDir=""
##download all data in athalRuns
sraObjects=[]
for x in athalRuns:
    